<p style="font-family: Arial; font-size:3.8em;color:purple; font-style:bold">
<br>FANAL-RECO VISUALIZATION<br>
</p>


This notebook illustrates the pseudo-reconstruction phase made in FANAL starting from MC data and finally providing a collection of voxels per event.

It provides the contents of the dataframes generated together with some control plots associated.

In [1]:
%matplotlib inline
%load_ext memory_profiler
%load_ext line_profiler

%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

FileNotFoundError: [Errno 2] No such file or directory: 'style-table.css'

#### Importing general stuff

In [ ]:
# General importings
import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

# Specific IC stuff
import invisible_cities.core.system_of_units as units

# Specific fanal stuff
from fanal.core.detector          import get_active_size
from fanal.core.fanal_types       import DetName
#from fanal.mc.mc_utilities        import print_mc_event

from fanal.fanal_reco             import fanal_reco


In [ ]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

# SETUP

In [ ]:
DET_NAME = "next100"
DET_NAME = getattr(DetName, DET_NAME)

RECO_FNAME = "../data/next100/bb0nu/reco/next100.bb0nu.fwhm_07.voxel_10x10x10.reco.h5"
#RECO_FNAME = "../data/next100/Bi214/reco/next100.Bi214.fwhm_07.voxel_10x10x10.reco.h5"
#RECO_FNAME = "../data/next100/Tl208/reco/next100.Tl208.fwhm_07.voxel_10x10x10.reco.h5"

#RECO_FNAME = "../data/next100/bb0nu/reco/next100.bb0nu.fwhm_05.voxel_3x3x3.reco.h5"
#RECO_FNAME = "../data/next100/Bi214/reco/next100.Bi214.fwhm_05.voxel_3x3x3.reco.h5"
#RECO_FNAME = "../data/next100/Tl208/reco/next100.Tl208.fwhm_05.voxel_3x3x3.reco.h5"

In [ ]:
with tb.open_file(RECO_FNAME, mode='r') as reco_file:
    reco_group_name = '/FANAL/' + reco_file.root.FANAL.__members__[0]

#### Verbosing the event counters

In [ ]:
with tb.open_file(RECO_FNAME, mode='r') as reco_file:
    simulated_events  = reco_file.get_node_attr(reco_group_name, 'simulated_events')
    stored_events     = reco_file.get_node_attr(reco_group_name, 'stored_events')
    smE_filter_events = reco_file.get_node_attr(reco_group_name, 'smE_filter_events')
    fid_filter_events = reco_file.get_node_attr(reco_group_name, 'fid_filter_events')

print('* Event counters ...')
print('''  Simulated events:  {0:9}
  Stored events:     {1:9}
  smE_filter events: {2:9}
  fid_filter events: {3:9}'''
      .format(simulated_events, stored_events, smE_filter_events, fid_filter_events))

#### Verbosing the dataframes

In [ ]:
events_df = pd.read_hdf(RECO_FNAME, reco_group_name + '/events')
voxels_df = pd.read_hdf(RECO_FNAME, reco_group_name + '/voxels')

In [ ]:
events_df_smE_True = events_df[events_df.smE_filter]
events_df_fid_True = events_df[events_df.fid_filter]

In [ ]:
voxels_df.head()

In [ ]:
events_df.head()

In [ ]:
events_df_smE_True.head()

In [ ]:
events_df_fid_True.head()

In [ ]:
voxels_df.head()

In [ ]:
#voxels_df.loc[7]
#voxels_df[voxels_df.negli]

#### Generating some histograms ...

Plotting event counters

In [ ]:
fig = plt.figure(figsize = (8,4))

counters_toPlot = {'Simulated Events':  simulated_events,
                   'Stored Events':     stored_events,
                   'smE_filter Events': smE_filter_events,
                   'fid_filter Events': fid_filter_events}

plt.bar(range(len(counters_toPlot)), list(counters_toPlot.values()), align='center',
        color=['b', 'r', 'c', 'y'], yerr=np.sqrt([simulated_events, stored_events,
                                                  smE_filter_events, fid_filter_events]))
plt.xticks(range(len(counters_toPlot)), list(counters_toPlot.keys()), rotation='30')
plt.show()

Plotting event energies (MC & Smeared) of every event

In [ ]:
fig = plt.figure(figsize = (12,5))
num_bins = 50

ax1 = fig.add_subplot(1, 2, 1)
plt.hist(events_df.mcE, num_bins, [2.3, 2.6])
#plt.hist(events_df.mcE, num_bins, [0., 2.6])
plt.xlabel('Event energy [MeV]', size=12)
plt.ylabel('Num. events', size=12)
plt.title('MonteCarlo Event Energy [MeV]')

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(events_df.smE, num_bins, [2.3, 2.6])
#plt.hist(events_df.smE, num_bins, [0., 2.6])
plt.xlabel('Event energy [MeV]', size=12)
plt.ylabel('Num. events', size=12)
plt.title('Smeared Event Energy [MeV]')


Plotting the number of voxels per event that passed the "Smeared Energy" filter

In [ ]:
events_df_smE_True['num_voxels'].value_counts().head(50).sort_index().plot(kind='bar', figsize=(8,6))

Plotting the voxels energies for all the events

In [ ]:
voxels_df[voxels_df.negli == False].hist('E', figsize =(8,4),
                                         bins = 50, range = (0, 0.2),
                                         grid = True, log = False,
                                         label = 'Voxel Energy', xlabelsize = 12,
                                         density = False, cumulative = False,
                                         histtype = 'stepfilled',
                                         # Type of Histogram ('bar', 'barstacked', 'step', 'stepfilled')
                                         orientation = 'vertical')


Plotting the spatial distribution of voxels from fiducial events

In [ ]:
ACTIVE_dimensions = get_active_size(DET_NAME)

#fid_voxels = voxels_df[voxels_df.event_indx.isin(events_df_fid_True.index)]
fid_voxels = voxels_df.loc[events_df_fid_True.index]

fig = plt.figure(figsize = (18,5))
num_bins = int(ACTIVE_dimensions.z_max/20)

ax1 = fig.add_subplot(1, 3, 1)
plt.hist(fid_voxels.X, num_bins, [-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad])
plt.title('X [mm]', size=14)

ax2 = fig.add_subplot(1, 3, 2)
plt.hist(fid_voxels.Y, num_bins, [-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad])
plt.title('Y [mm]', size=14)

ax3 = fig.adax1 = fig.add_subplot(1, 3, 3)
plt.hist(fid_voxels.Z, num_bins, [ACTIVE_dimensions.z_min, ACTIVE_dimensions.z_max])
plt.title('Z [mm]', size=14)

In [ ]:
from   matplotlib.colors import LogNorm

fig = plt.figure(figsize = (18,5.5))
num_bins = int(ACTIVE_dimensions.z_max/20)

ax1 = fig.add_subplot(1, 3, 1)
plt.hist2d(fid_voxels.X, fid_voxels.Y, num_bins, [[-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad],
                                                  [-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad]],
           norm=LogNorm())
plt.xlabel('X [mm]', size=12)
plt.ylabel('Y [mm]', size=12)
plt.title('X-Y [mm]', size=14)

ax2 = fig.add_subplot(1, 3, 2)
plt.hist2d(fid_voxels.X, fid_voxels.Z, num_bins, [[-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad],
                                                  [ACTIVE_dimensions.z_min, ACTIVE_dimensions.z_max]],
           norm=LogNorm())
plt.xlabel('X [mm]', size=12)
plt.ylabel('Z [mm]', size=12)
plt.title('X-Z [mm]', size=14)

ax3 = fig.adax1 = fig.add_subplot(1, 3, 3)
plt.hist2d(fid_voxels.Y, fid_voxels.Z, num_bins, [[-ACTIVE_dimensions.rad, ACTIVE_dimensions.rad],
                                                  [ACTIVE_dimensions.z_min, ACTIVE_dimensions.z_max]],
           norm=LogNorm())
plt.xlabel('Y [mm]', size=12)
plt.ylabel('Z [mm]', size=12)
plt.title('Y-Z [mm]', size=14)


Plotting the histogram of voxel sizes ...

In [ ]:
fig = plt.figure(figsize = (18,5))
num_bins = 45

ax1 = fig.add_subplot(1, 3, 1)
plt.hist(events_df_fid_True.voxel_sizeX, num_bins, [0, 15])
plt.title('SizeX [mm]', size=14)

ax1 = fig.add_subplot(1, 3, 2)
plt.hist(events_df_fid_True.voxel_sizeY, num_bins, [0, 15])
plt.title('SizeY [mm]', size=14)

ax1 = fig.add_subplot(1, 3, 3)
plt.hist(events_df_fid_True.voxel_sizeZ, num_bins, [0, 15])
plt.title('SizeZ [mm]', size=14)

In [ ]:
events_df_smE_True.hist(['mcE', 'smE'], bins=20,  figsize = (16,6), range = [2.4, 2.5])

In [ ]:
fig = plt.figure(figsize = (10,10))
num_bins = 50

ax1 = fig.add_subplot(2, 1, 1)
plt.hist(events_df_smE_True.mcE, num_bins, [2.39,2.51])
plt.xlabel('Total energy (MeV)')
ax1 = fig.add_subplot(2, 1, 2)
plt.hist(events_df_smE_True.smE, num_bins, [2.39,2.51])
plt.xlabel('Total energy (MeV)')

from scipy.stats import norm
(mu,sigma) = norm.fit(events_df[((events_df.smE>2.445) & (events_df.smE<2.475))].smE)
print('mean: {0:.4f} MeV  sigma: {1:.4f} MeV'.format(mu, sigma))

In [ ]:
len(voxels_df)

In [ ]:
sum(events_df.smE_filter)

In [ ]:
#event_to_print = 100000462

#with tb.open_file(RECO_FNAME, mode='r') as reco_file:
#    files_list = reco_file.get_node_attr(reco_group_name, 'input_sim_files')
    
#print_mc_event(event_to_print, files_list, with_hits=True)

In [ ]:
#from fanal.mc.mc_io_functions  import load_mc_particles
#file_mcParts = load_mc_particles('/Users/Javi/Development/nexus/tst.next.h5')

In [ ]:
#fiducialEvt_mcParts = file_mcParts.loc[events_df_fid_True.index]
#fiducialEvt_mcParts[(fiducialEvt_mcParts.mother_id == 1) & 
#                    (fiducialEvt_mcParts.name == 'e-')].final_volume.value_counts()
